# ***PyMongo***

In [4]:
# Prerequisies
import pymongo

In [5]:
# Making a connection with MongoClient
from pymongo import MongoClient
client = MongoClient("localhost", 27017)

In [6]:
# Getting a database
db = client["test-database"]

In [7]:
# Getting a collection
collection = db["test-collecttion"]

In [8]:
# blog post
import datetime
post={
    "author": "Yash",
    "text": "My first blog post",
    "tags": ["mongodb","python", "pymongo"],
    "date": datetime.datetime.now(tz=datetime.timezone.utc),
}

In [9]:
# inserting a document
posts = db.posts

In [10]:
post_id = posts.insert_one(post).inserted_id

In [11]:
post_id

ObjectId('64bcb71e31b116af8bd08870')

In [12]:
db.list_collection_names()

['posts']

In [13]:
# Getting a single document with
import pprint
pprint.pprint(posts.find_one())

{'_id': ObjectId('64bcb71e31b116af8bd08870'),
 'author': 'Yash',
 'date': datetime.datetime(2023, 7, 23, 5, 14, 5, 821000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post'}


In [14]:
pprint.pprint(posts.find_one({"author": "Yash"}))

{'_id': ObjectId('64bcb71e31b116af8bd08870'),
 'author': 'Yash',
 'date': datetime.datetime(2023, 7, 23, 5, 14, 5, 821000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post'}


In [15]:
# no result
posts.find_one({"author": "Eliot"})

In [16]:
# Querying by  ObjectId
post_id

ObjectId('64bcb71e31b116af8bd08870')

In [17]:
pprint.pprint(posts.find_one({"_id": post_id}))

{'_id': ObjectId('64bcb71e31b116af8bd08870'),
 'author': 'Yash',
 'date': datetime.datetime(2023, 7, 23, 5, 14, 5, 821000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post'}


In [18]:
# convert the objectid from a string
from bson.objectid import ObjectId
# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

In [19]:
# Bulk inserts
new_posts = [
    {
        "author":"David",
        "text": "Another post",
        "tags": ["bulk","insert"],
        "date": datetime.datetime(2009, 11, 12, 11, 14),
    },
    {
        "author": "Sam",
        "title": "Mongodb",
        "text": "most used tool in industry",
        "date": datetime.datetime(2009, 11, 10, 10, 45),
    },
]

In [20]:
result = posts.insert_many(new_posts)

In [21]:
result.inserted_ids

[ObjectId('64bcb80331b116af8bd08871'), ObjectId('64bcb80331b116af8bd08872')]

The result from insert_many() now returns two ObjectId instances, one for each inserted document.

new_posts[1] has a different “shape” than the other posts - there is no "tags" field and we’ve added a new field, "title". This is what we mean when we say that MongoDB is schema-free.

In [22]:
# querying for more than one document
for post in posts.find():
    pprint.pprint(post)

{'_id': ObjectId('64bcb71e31b116af8bd08870'),
 'author': 'Yash',
 'date': datetime.datetime(2023, 7, 23, 5, 14, 5, 821000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post'}
{'_id': ObjectId('64bcb80331b116af8bd08871'),
 'author': 'David',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post'}
{'_id': ObjectId('64bcb80331b116af8bd08872'),
 'author': 'Sam',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'most used tool in industry',
 'title': 'Mongodb'}


In [23]:
# Just like we did with find_one(), we can pass a document to find() to limit the returned results.
for post in posts.find({"author": "Yash"}):
    pprint.pprint(post)

{'_id': ObjectId('64bcb71e31b116af8bd08870'),
 'author': 'Yash',
 'date': datetime.datetime(2023, 7, 23, 5, 14, 5, 821000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post'}


In [24]:
# counting
# If we just want to know how many documents match a query we can perform a count_documents() operation instead of a full query. We can get a count of all of the documents in a collection
posts.count_documents({})

3

In [26]:
posts.count_documents({"author":"sam"})

0

In [27]:
# Range queries
# Lets perform a query where we limit results to posts older than a certain date, but also sort the results by author
d = datetime.datetime(2009, 11, 12, 12)
for post in posts.find({"date": {"$lt": d}}).sort("author"):
    pprint.pprint(post)

{'_id': ObjectId('64bcb80331b116af8bd08871'),
 'author': 'David',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post'}
{'_id': ObjectId('64bcb80331b116af8bd08872'),
 'author': 'Sam',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'most used tool in industry',
 'title': 'Mongodb'}


In [28]:
# Indexing
result = db.profiles.create_index([("user_id", pymongo.ASCENDING)], unique=True)
sorted(list(db.profiles.index_information()))

['_id_', 'user_id_1']

In [29]:
user_profiles = [{"user_id": 211, "name": "Luke"}, {"user_id": 212, "name": "Manju"}]
result = db.profiles.insert_many(user_profiles)

In [30]:
# The index prevents us from inserting a document whose user_id is already in the collection:
new_profile = {"user_id": 213, "name": "Drew"}
duplicate_profile = {"user_id": 212, "name": "Tommy"}
result = db.profiles.insert_one(new_profile)  # This is fine.
result = db.profiles.insert_one(duplicate_profile)

DuplicateKeyError: E11000 duplicate key error collection: test-database.profiles index: user_id_1 dup key: { : 212 }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: test-database.profiles index: user_id_1 dup key: { : 212 }'}